In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import netCDF4 as nc4 
import numpy as np
import pandas as pd
import xarray as xr
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil
# import pyviscous


In [7]:
def NSE(s,o):
    """
    Created on Thu Jan 20 15:36:37 2011
    @ author:                  Sat Kumar Tomer
    @ author's webpage:        http://civil.iisc.ernet.in/~satkumar/
    @ author's email id:       satkumartomer@gmail.com
    @ author's website:        www.ambhas.com

    Nash Sutcliffe efficiency coefficient
    input:
        s: simulated [time]
        o: observed [time]
    output:
        ns: Nash Sutcliffe efficient coefficient
    """
    s,o = filter_nan(s,o)
    return 1 - sum((s-o)**2)/sum((o-np.mean(o))**2)

def RMSE(s,o):
    """
    Created on Thu Jan 20 15:36:37 2011
    @ author:                  Sat Kumar Tomer
    @ author's webpage:        http://civil.iisc.ernet.in/~satkumar/
    @ author's email id:       satkumartomer@gmail.com
    @ author's website:        www.ambhas.com

    Nash Sutcliffe efficiency coefficient
    input:
        s: simulated [time]
        o: observed [time]
    output:
        ns: Nash Sutcliffe efficient coefficient
    """
    s,o = filter_nan(s,o)
    return ((sum((s-o)**2)/len(o))**0.5 )

def filter_nan(s,o):
    """
    Created on Thu Jan 20 15:36:37 2011
    @ author:                  Sat Kumar Tomer
    @ author's webpage:        http://civil.iisc.ernet.in/~satkumar/
    @ author's email id:       satkumartomer@gmail.com
    @ author's website:        www.ambhas.com

    this functions removed the data  from simulated and observed data
    whereever the observed data contains nan

    this is used by all other functions, otherwise they will produce nan as
    output
    """
    data = np.array([s.flatten(),o.flatten()])
    data = np.transpose(data)
    data = data[~np.isnan(data).any(1)]
    return data[:,0],data[:,1]


In [8]:
#
# param = np.loadtxt('../mizuRoute_output_all/params.txt')
# obj_par = pd.DataFrame(param, columns=['HYP_Erate_emr',\
#                                        'HYP_Qrate_emr',\
#                                        'scale_factor_Ep_temp',\
#                                        'scale_factor_P_temp'])
obj_par = pd.read_csv("../mizuRoute_output_all/params.csv")

obj_par ['1_NSE'] = np.nan
obj_par ['RMSE'] = np.nan


# read the Dahitti data set
lake_elevation = pd.read_csv('../data/DAHITI_Data/Lake_Victoria.csv')
lake_elevation['date']  = pd.DatetimeIndex(lake_elevation['date'])
lake_elevation = lake_elevation.set_index('date')


file_names = sorted(glob.glob('../mizuRoute_output_all/sim_0*.nc'))

m = 0

for file_name in file_names:
    
    print(m)
    
    ds = xr.open_dataset(file_name)
    
    ds['ele_sim'] = ds['IRFvolume'].copy()
    ds['ele_sim'][:] = ds['ele_sim'][:]/(68800*10**6)
    ds['ele_sim'][:] = ds['ele_sim'][:] + 1122.86
    
    # calculate objective function
    df = ds['ele_sim'][:,1].to_dataframe()
    lake_elevation_reindex = lake_elevation.copy()
    lake_elevation_reindex = lake_elevation_reindex.reindex(df.index)
    
    print(lake_elevation_reindex)
    print(df)
    
    df['ele_obs'] = np.nan
    df['ele_obs'] = lake_elevation_reindex['water_level'].values
    
    #
    obj_par['1_NSE'].iloc[m] = 1 - NSE(df['ele_sim'].values,df['ele_obs'].values)
    obj_par['RMSE'].iloc[m] = RMSE(df['ele_sim'].values,df['ele_obs'].values)
    
    
    m = m + 1
    
obj_par.to_csv('../mizuRoute_output_all/obj_par.csv')


0
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.850464
2000-01-02  1135.847412
2000-01-03  1135.846313
2000-01-04  1135.842896
2000-01-05  1135.838623
...                 ...
2013-12-27  1139.905396
2013-12-28  1139.902100
2013-12-29  1139.901245
2013-12-30  1139.910278
2013-12-31  1139.926147

[5114 rows x 1 columns]
1
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...           

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.850708
2000-01-02  1135.848022
2000-01-03  1135.848511
2000-01-04  1135.846069
2000-01-05  1135.842285
...                 ...
2013-12-27  1140.285400
2013-12-28  1140.281006
2013-12-29  1140.279053
2013-12-30  1140.291992
2013-12-31  1140.313599

[5114 rows x 1 columns]
12
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...            

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.840210
2000-01-02  1135.827271
2000-01-03  1135.817383
2000-01-04  1135.804443
2000-01-05  1135.790527
...                 ...
2013-12-27  1133.963989
2013-12-28  1133.959839
2013-12-29  1133.958130
2013-12-30  1133.971313
2013-12-31  1133.992798

[5114 rows x 1 columns]
24
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...            

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.834961
2000-01-02  1135.816528
2000-01-03  1135.800293
2000-01-04  1135.781982
2000-01-05  1135.763062
...                 ...
2013-12-27  1133.409302
2013-12-28  1133.405762
2013-12-29  1133.404419
2013-12-30  1133.412842
2013-12-31  1133.427979

[5114 rows x 1 columns]
36
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...            

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.838135
2000-01-02  1135.822876
2000-01-03  1135.810669
2000-01-04  1135.795654
2000-01-05  1135.779663
...                 ...
2013-12-27  1133.788208
2013-12-28  1133.784302
2013-12-29  1133.782837
2013-12-30  1133.794678
2013-12-31  1133.814453

[5114 rows x 1 columns]
48
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...            

...

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.847290
2000-01-02  1135.840942
2000-01-03  1135.836548
2000-01-04  1135.829834
2000-01-05  1135.822388
...                 ...
2013-12-27  1133.934937
2013-12-28  1133.931030
2013-12-29  1133.929565
2013-12-30  1133.937500
2013-12-31  1133.951904

[5114 rows x 1 columns]
948
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...           

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.846191
2000-01-02  1135.839111
2000-01-03  1135.834595
2000-01-04  1135.827148
2000-01-05  1135.818848
...                 ...
2013-12-27  1134.363403
2013-12-28  1134.359009
2013-12-29  1134.357300
2013-12-30  1134.368652
2013-12-31  1134.387329

[5114 rows x 1 columns]
960
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...           

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.848389
2000-01-02  1135.843384
2000-01-03  1135.840942
2000-01-04  1135.835815
2000-01-05  1135.829712
...                 ...
2013-12-27  1136.136230
2013-12-28  1136.131958
2013-12-29  1136.130249
2013-12-30  1136.141357
2013-12-31  1136.160278

[5114 rows x 1 columns]
972
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...           

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.852661
2000-01-02  1135.851807
2000-01-03  1135.853271
2000-01-04  1135.852295
2000-01-05  1135.850342
...                 ...
2013-12-27  1148.892944
2013-12-28  1148.890991
2013-12-29  1148.891235
2013-12-30  1148.902588
2013-12-31  1148.921265

[5114 rows x 1 columns]
984
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...           

            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...                 ...    ...
2013-12-27          NaN    NaN
2013-12-28          NaN    NaN
2013-12-29          NaN    NaN
2013-12-30     1135.294    0.0
2013-12-31          NaN    NaN

[5114 rows x 2 columns]
                ele_sim
time                   
2000-01-01  1135.852051
2000-01-02  1135.850586
2000-01-03  1135.851562
2000-01-04  1135.849976
2000-01-05  1135.847534
...                 ...
2013-12-27  1142.733765
2013-12-28  1142.730225
2013-12-29  1142.729004
2013-12-30  1142.738892
2013-12-31  1142.756348

[5114 rows x 1 columns]
996
            water_level  error
time                          
2000-01-01          NaN    NaN
2000-01-02          NaN    NaN
2000-01-03          NaN    NaN
2000-01-04          NaN    NaN
2000-01-05          NaN    NaN
...           